The present document has been made to generate a CSV file of social media usage to be exported to PowerBI.

In [4]:
#Import Libraries
import pandas as pd
import numpy as np



1st step. Understand the data to be generated for its use in the model. 

Possible Tables: 

Date Period (Months) 

x 

Channel (programmatic, paid search, paid social, organic) 

x Data Source (Amazon Ad Server, Stack Adapt, ...) 

x Campaign (Business focused, Persistent 24/7, ...)


*The tables to be made were based on the visuals to be generated.


Tables: 

1. Dates Table

Date period: 1 year 
Granularity: Months 


2. Channels *4 mentioned *

Programmatic, Paid Search, Paid Social, Organic 


3. Data Source 

Examples: *from the Data Source table *
Amazon Ad Server, StackAdapt, LinkedIn Ads, Facebook, Google Display & Video 360, Bing Ads, Google Search Ads


4. Campaign

Examples: *from the campaign visual table. 

Business-Focused zero tolerance
Persistent 24/7 attitude 
Integrated dedicated contingency 
Profound intangible policy 
Centralized modular throughput 
Automated uniform software 
Cross-platform static hierarchy 
Networked value-added time-frame 

In [5]:
#Make a dummy date period table 
months = pd.DataFrame({
    "date": pd.date_range(start="2023-01-01", end="2023-12-01", freq="MS")
})
display(months)

,date
0,2023-01-01
1,2023-02-01
2,2023-03-01
3,2023-04-01
4,2023-05-01
5,2023-06-01
6,2023-07-01
7,2023-08-01
8,2023-09-01
9,2023-10-01


In [6]:
#Make a dummy channel table
channels = pd.DataFrame({
    'channel': ["Programmatic", "Paid Search", "Paid Social", "Organic"]})
channels['channel_id'] = range(1, len(channels) + 1)
display(channels)


,channel,channel_id
0,Programmatic,1
1,Paid Search,2
2,Paid Social,3
3,Organic,4


In [7]:
#Make a dummy Data Source table 
data_sources = pd.DataFrame({
    'data_sources': [
"Amazon Ad Server", "StackAdapt", "LinkedIn Ads", "Facebook", "Google Display & Video 360", "Bing Ads", "Google Search Ads"
]})

#connection with channels table
channel_map = {
    "Amazon Ad Server": 1,        # Programmatic
    "StackAdapt": 1,              
    "Google Display & Video 360": 1,

    "Google Search Ads": 2,        # Paid Search
    "Bing Ads": 2,

    "Facebook": 3,                # Paid Social
    "LinkedIn Ads": 3
}
data_sources["data_source_id"] = range(1, len(data_sources) + 1)
data_sources["channel_id"] = data_sources["data_sources"].map(channel_map)
display(data_sources)

,data_sources,data_source_id,channel_id
0,Amazon Ad Server,1,1
1,StackAdapt,2,1
2,LinkedIn Ads,3,3
3,Facebook,4,3
4,Google Display & Video 360,5,1
5,Bing Ads,6,2
6,Google Search Ads,7,2


In [21]:
#Make a dummy Campaign table 
campaigns = pd.DataFrame({
    'campaign': ["Business-Focused zero tolerance",
"Persistent 24/7 attitude", "Integrated dedicated contingency", "Profound intangible policy", 
"Centralized modular throughput", "Automated uniform software", 
"Cross-platform static hierarchy", "Networked value-added time-frame"]})
campaigns['campaign_id'] = range(1, len(campaigns) + 1)


campaigns["data_source_id"] = data_sources["data_source_id"].values[
    campaigns.index % len(data_sources)
]
display(campaigns)

,campaign,campaign_id,data_source_id
0,Business-Focused zero tolerance,1,1
1,Persistent 24/7 attitude,2,2
2,Integrated dedicated contingency,3,3
3,Profound intangible policy,4,4
4,Centralized modular throughput,5,5
5,Automated uniform software,6,6
6,Cross-platform static hierarchy,7,7
7,Networked value-added time-frame,8,1


Dummy tables generated

Next, 

make the Facts table


In [22]:
#Facts table
impression_trends_millions = {
    1: [6.0, 5.6, 6.2, 6.0, 6.5, 6.3, 6.8, 6.6, 6.9, 6.95, 6.98, 7.0],  # Programmatic
    2: [3.0, 2.8, 2.6, 8.0, 3.5, 3.0, 3.0, 3.2, 2.6, 8.0, 6.5, 7.0],  # Paid Search
    3: [1.0, 1.5, 2.2, 3.0, 2.5, 2.7, 3.0, 3.0, 0.8, 3.1, 2.0, 2.5],  # Paid Social
    4: [0]*12                                                       # Organic
}


channel_metrics = {
    1: {"ctr": 0.002, "cpc": 2.0, "cvr": 0.005},   # Programmatic
    2: {"ctr": 0.035, "cpc": 3.5, "cvr": 0.04},    # Paid Search
    3: {"ctr": 0.009, "cpc": 1.5, "cvr": 0.015},   # Paid Social
    4: {"ctr": 0.05,  "cpc": 0.0, "cvr": 0.08}     # Organic
}

fact_rows = []

for month_idx, date in enumerate(months["date"]):

    for _, ch in channels.iterrows():
        channel_id = ch["channel_id"]

        total_impressions = int(
            impression_trends_millions[channel_id][month_idx] * 1_000_000
        )

        metrics = channel_metrics[channel_id]

        # Channel → Data Sources
        sources = data_sources[
            data_sources["channel_id"] == channel_id
        ]

        if sources.empty:
            continue

        impressions_per_source = (
            total_impressions // len(sources)
            if total_impressions > 0 else 0
        )

        for _, src in sources.iterrows():
            data_source_id = src["data_source_id"]

            # Data Source → Campaigns
            source_campaigns = campaigns[
                campaigns["data_source_id"] == data_source_id
            ]

            if source_campaigns.empty:
                continue

            impressions_per_campaign = (
                impressions_per_source // len(source_campaigns)
                if impressions_per_source > 0 else 0
            )

            for _, camp in source_campaigns.iterrows():
                impressions = impressions_per_campaign
                clicks = int(impressions * metrics["ctr"])
                spend = round(clicks * metrics["cpc"], 2)
                conversions = int(clicks * metrics["cvr"])
                video_views = int(impressions * 0.25) if channel_id != 2 else 0

                fact_rows.append({
                    "date": date,
                    "channel_id": channel_id,
                    "data_source_id": data_source_id,
                    "campaign_id": camp["campaign_id"],
                    "impressions": impressions,
                    "clicks": clicks,
                    "spend": spend,
                    "video_views": video_views,
                    "conversions": conversions
                })
fact_marketing = pd.DataFrame(fact_rows)
display(fact_marketing)

,date,channel_id,data_source_id,campaign_id,impressions,clicks,spend,video_views,conversions
0,2023-01-01,1,1,1,1000000,2000,4000.0,250000,10
1,2023-01-01,1,1,8,1000000,2000,4000.0,250000,10
2,2023-01-01,1,2,2,2000000,4000,8000.0,500000,20
3,2023-01-01,1,5,5,2000000,4000,8000.0,500000,20
4,2023-01-01,2,6,6,1500000,52500,183750.0,0,2100
...,...,...,...,...,...,...,...,...,...
91,2023-12-01,1,5,5,2333333,4666,9332.0,583333,23
92,2023-12-01,2,6,6,3500000,122500,428750.0,0,4900
93,2023-12-01,2,7,7,3500000,122500,428750.0,0,4900
94,2023-12-01,3,3,3,1250000,11250,16875.0,312500,168


Facts table generated

Next, 

Quality check. Any nulls? any empty data row or column? 

In [27]:
months.info()
print() #for adding a space between outputs
channels.info()
print() 
data_sources.info() 
print() 
campaigns.info()
print() 
fact_marketing.info()

print("Dataframes created successfully. No null values found.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    12 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 228.0 bytes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   channel     4 non-null      object
 1   channel_id  4 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 196.0+ bytes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   data_sources    7 non-null      object
 1   data_source_id  7 non-null      int64 
 2   channel_id      7 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 300.0+ bytes

<class 'pandas.core.f

Quality checked. 

Next, 

Export to CSV files to be connected in Power BI. 

In [28]:
#Files to be exported: 1. months, 2. channels, 3. data_sources, 4. campaigns, 5. fact_marketing
months.to_csv("months.csv", index=False)
channels.to_csv("channels.csv", index=False)
data_sources.to_csv("data_sources.csv", index=False)
campaigns.to_csv("campaigns.csv", index=False)
fact_marketing.to_csv("fact_marketing.csv", index=False)

The objective of the python notebook was completed successfully. 

1. Dummy data generated. 

2. CSV files generated ready to set connection with power BI to make the marketing dashboard. 